# React JS Advanced Topics


# Higher Order Components

* Higher-order components (HOCs) are a powerful feature of the React library. They allow you to reuse component logic across multiple components.



## Benefits of Using Higher-Order Components in React

* Reusability
  * HOCs allow you to reuse component logic across multiple components, 
    * which can save time and reduce code duplication.
* Flexibility
  * HOCs can take additional arguments, which allows you to customize the behavior of the HOC. 
  * This makes them a flexible way to add functionality to your components.
* Separation of concerns
  * HOCs can help separate concerns in your code by encapsulating certain functionality in a separate component.
* Composition
  * HOCs can be composed together to create more complex functionality. 
  * This allows you to build up functionality from smaller, reusable pieces.
* Higher-order components can be used to 
  * implement cross-cutting concerns in your application 
  * such as 
    * authentication, 
    * error handling, 
    * logging, 
    * performance tracking, and 
    * many other features.



## Higher-Order Component Structure

* To define a Higher-Order Component (HOC) 

1. First, you'll define the HOC function. 
  * This is a function 
    * that takes a component as input and 
    * returns a new component with additional functionality.

```js
const hoc = (WrappedComponent) => {
  // ...
}
```

2. Then you define the new component. 
   * This is a class component that wraps the WrappedComponent and adds additional functionality.

```js
class NewComponent extends React.Component {
  // ...
  render() {
    // ...
  }
}   
```

3. Next, you pass props to the `WrappedComponent`. 
   * In the `render() method` of the `NewComponent` , 
     * pass all the props (including the additional props added by the HOC) to the `WrappedComponent`.

```js
render() {
  return <WrappedComponent {...this.props} additionalProp={additionalProp} />
}
```
4. Finally, return the new component. 
   * The HOC function should return the NewComponent 
     * so it can be used in the application.

```js
const hoc = (WrappedComponent) => {
  class NewComponent extends React.Component {
    // ...
    render() {

      // ...
    }
  }

  return NewComponent;
}
```

## When to Use HOCs in your React Code
* Authentication
* Logging
* Styling and Theming

## How to Use Higher-Order Components in React

* Here's an example of a simple HOC:

```js

import React from 'react';
import ReactDOM from 'react-dom';
const withLoading = (WrappedComponent) => {
  class WithLoading extends React.Component {
    state = {
      isLoading: true,
    };

    componentDidMount() {
      setTimeout(() => {
        this.setState({ isLoading: false });
      }, 2000);
    }

    render() {
      return (
        <WrappedComponent
          {...this.props}
          loading={this.state.isLoading}
        />
      );
    }
  }

  WithLoading.displayName = `withLoading(${WrappedComponent.displayName || WrappedComponent.name})`;

  return WithLoading;
};

const MyComponent = ({ loading }) => (
  <div>
    {loading ? <p>Loading...</p> : <p>Hello, world!</p>}
  </div>
);

const MyComponentWithLoading = withLoading(MyComponent);

ReactDOM.render(
  <MyComponentWithLoading />,
  document.getElementById("root")
);

```

# Render Props

* The term “render prop” refers 
  * to a technique for sharing code between React components 
  * using a prop whose value is a function.

* A component with a render prop 
  * takes a function 
    * that returns a React element and 
    * calls it instead of implementing its own render logic.

```js

<DataProvider render={data => (
  <h1>Hello {data.target}</h1>
)}/>

```


## Use Render Props for Cross-Cutting Concerns
Components are the primary unit of code reuse in React, but it’s not always obvious how to share the state or behavior that one component encapsulates to other components that need that same state.

For example, the following component tracks the mouse position in a web app:

```js
class MouseTracker extends React.Component {
  constructor(props) {
    super(props);
    this.handleMouseMove = this.handleMouseMove.bind(this);
    this.state = { x: 0, y: 0 };
  }

  handleMouseMove(event) {
    this.setState({
      x: event.clientX,
      y: event.clientY
    });
  }

  render() {
    return (
      <div style={{ height: '100vh' }} onMouseMove={this.handleMouseMove}>
        <h1>Move the mouse around!</h1>
        <p>The current mouse position is ({this.state.x}, {this.state.y})</p>
      </div>
    );
  }
}
```



* As the cursor moves around the screen, 
  * the component displays its (x, y) coordinates in a <p>.

* Now the question is: 
  * How can we reuse this behavior in another component? In other words, 
  * if another component needs to know about the cursor position, 
  * can we encapsulate that behavior so that we can easily share it with that component?

* Since components are the basic unit of code reuse in React, 
  * let’s try refactoring the code a bit to use a `<Mouse>` component 
  * that encapsulates the behavior we need to reuse elsewhere.

```js
// The <Mouse> component encapsulates the behavior we need...
class Mouse extends React.Component {
  constructor(props) {
    super(props);
    this.handleMouseMove = this.handleMouseMove.bind(this);
    this.state = { x: 0, y: 0 };
  }

  handleMouseMove(event) {
    this.setState({
      x: event.clientX,
      y: event.clientY
    });
  }

  render() {
    return (
      <div style={{ height: '100vh' }} onMouseMove={this.handleMouseMove}>

        {/* ...but how do we render something other than a <p>? */}
        <p>The current mouse position is ({this.state.x}, {this.state.y})</p>
      </div>
    );
  }
}

class MouseTracker extends React.Component {
  render() {
    return (
      <>
        <h1>Move the mouse around!</h1>
        <Mouse />
      </>
    );
  }
}
```



* Now the `<Mouse>` component encapsulates all behavior associated with listening for mousemove events and storing the (x, y) position of the cursor, but it’s not yet truly reusable.

* For example, 
  * let’s say we have a `<Cat>` component 
  * that renders the image of a cat chasing the mouse around the screen. 
* We might use a `<Cat mouse={{ x, y }}>` prop to tell the component the coordinates of the mouse so it knows where to position the image on the screen.

* As a first pass, 
  * you might try rendering the `<Cat>` inside `<Mouse>’s` render method, like this:

```js

class Cat extends React.Component {
  render() {
    const mouse = this.props.mouse;
    return (
      <img src="/cat.jpg" style={{ position: 'absolute', left: mouse.x, top: mouse.y }} />
    );
  }
}

class MouseWithCat extends React.Component {
  constructor(props) {
    super(props);
    this.handleMouseMove = this.handleMouseMove.bind(this);
    this.state = { x: 0, y: 0 };
  }

  handleMouseMove(event) {
    this.setState({
      x: event.clientX,
      y: event.clientY
    });
  }

  render() {
    return (
      <div style={{ height: '100vh' }} onMouseMove={this.handleMouseMove}>

        {/*
          We could just swap out the <p> for a <Cat> here ... but then
          we would need to create a separate <MouseWithSomethingElse>
          component every time we need to use it, so <MouseWithCat>
          isn't really reusable yet.
        */}
        <Cat mouse={this.state} />
      </div>
    );
  }
}

class MouseTracker extends React.Component {
  render() {
    return (
      <div>
        <h1>Move the mouse around!</h1>
        <MouseWithCat />
      </div>
    );
  }
}
```



* This approach will work for our specific use case, 
  * but we haven’t achieved the objective of truly encapsulating the behavior in a reusable way. 
* Now, 
  * every time we want the mouse position for a different use case, 
  * we have to create a new component (i.e. essentially another `<MouseWithCat>`) 
  * that renders something specifically for that use case.

* Here’s where the render prop comes in: 
  * Instead of hard-coding a `<Cat>` inside a `<Mouse>` component, 
  * and effectively changing its rendered output, 
  * we can provide `<Mouse>` with a function prop that 
  * it uses to dynamically determine what to render–a render prop.

```js
class Cat extends React.Component {
  render() {
    const mouse = this.props.mouse;
    return (
      <img src="/cat.jpg" style={{ position: 'absolute', left: mouse.x, top: mouse.y }} />
    );
  }
}

class Mouse extends React.Component {
  constructor(props) {
    super(props);
    this.handleMouseMove = this.handleMouseMove.bind(this);
    this.state = { x: 0, y: 0 };
  }

  handleMouseMove(event) {
    this.setState({
      x: event.clientX,
      y: event.clientY
    });
  }

  render() {
    return (
      <div style={{ height: '100vh' }} onMouseMove={this.handleMouseMove}>

        {/*
          Instead of providing a static representation of what <Mouse> renders,
          use the `render` prop to dynamically determine what to render.
        */}
        {this.props.render(this.state)}
      </div>
    );
  }
}

class MouseTracker extends React.Component {
  render() {
    return (
      <div>
        <h1>Move the mouse around!</h1>
        <Mouse render={mouse => (
          <Cat mouse={mouse} />
        )}/>
      </div>
    );
  }
}
```



* Now, 
  * instead of 
    * effectively cloning the `<Mouse>` component and 
    * hard-coding something else in its render method to solve for a specific use case, 
  * we provide a render prop 
    * that `<Mouse>` can use to dynamically determine what it renders.

* More concretely, 
  * a render prop is a function prop 
  * that a component uses to know what to render.

* This technique makes the behavior that we need to share extremely portable. 
* To get that behavior, 
  * render a `<Mouse>` with a render prop 
  * that tells it what to render with the current (x, y) of the cursor.

* One interesting thing to note about render props is that 
  * you can implement most higher-order components (HOC) using a regular component with a render prop. 
* For example, 
  * if you would prefer to have a withMouse HOC instead of a `<Mouse>` component, 
  * you could easily create one using a regular `<Mouse>` with a render prop:

```js

// If you really want a HOC for some reason, you can easily
// create one using a regular component with a render prop!
function withMouse(Component) {
  return class extends React.Component {
    render() {
      return (
        <Mouse render={mouse => (
          <Component {...this.props} mouse={mouse} />
        )}/>
      );
    }
  }
}
```

* So using a render prop makes it possible to use either pattern.



## Using Props Other Than render
* It’s important to remember 
  * that just because the pattern is called “render props” 
  * you don’t have to use a prop named render to use this pattern. 
* In fact, 
  * any prop 
    * that is a function 
    * that a component uses to know what to render is technically a “render prop”.

* Although the examples above use render, we could just as easily use the children prop!

```js
<Mouse children={mouse => (
  <p>The mouse position is {mouse.x}, {mouse.y}</p>
)}/>
```

* And remember, 
  * the children prop doesn’t actually need to be named in the list of “attributes” in your JSX element. 
* Instead, you can put it directly inside the element!

```js
<Mouse>
  {mouse => (
    <p>The mouse position is {mouse.x}, {mouse.y}</p>
  )}
</Mouse>
```

* You’ll see this technique used in the react-motion API.

* Since this technique is a little unusual, 
  * you’ll probably want to explicitly state that 
    * children should be a function in your propTypes when designing an API like this.


```js

Mouse.propTypes = {
  children: PropTypes.func.isRequired
};

```



## Caveats
* Be careful when using Render Props with `React.PureComponent`
* Using a render prop can negate the advantage 
  * that comes from using `React.PureComponent` 
  * if you create the function inside a render method. 
* This is because 
  * the shallow prop comparison will always return false for new props, 
  * and each render in this case will generate a new value for the render prop.

* For example, continuing with our `<Mouse>` component from above, 
  * if Mouse were to extend `React.PureComponent` instead of `React.Component`, 
  * our example would look like this:


```js

class Mouse extends React.PureComponent {
  // Same implementation as above...
}

class MouseTracker extends React.Component {
  render() {
    return (
      <div>
        <h1>Move the mouse around!</h1>

        {/*
          This is bad! The value of the `render` prop will
          be different on each render.
        */}
        <Mouse render={mouse => (
          <Cat mouse={mouse} />
        )}/>
      </div>
    );
  }
}
```
* In this example, 
  * each time `<MouseTracker>` renders, 
  * it generates a new function as the value of the `<Mouse render>` prop, 
  * thus negating the effect of `<Mouse>` extending `React.PureComponent` in the first place!

* To get around this problem, you can sometimes define the prop as an instance method, like so:

```js

class MouseTracker extends React.Component {
  // Defined as an instance method, `this.renderTheCat` always
  // refers to *same* function when we use it in render
  renderTheCat(mouse) {
    return <Cat mouse={mouse} />;
  }

  render() {
    return (
      <div>
        <h1>Move the mouse around!</h1>
        <Mouse render={this.renderTheCat} />
      </div>
    );
  }
}
```

* In cases where you cannot define the prop statically 
  * (e.g. because you need to close over the component’s props and/or state) 
  * <Mouse> should extend React.Component instead.

# Error Boundaries


# Portal & context


## Portal


## Context


# API calls


## Fetch


## Axios


# Refs


# Hooks

* Hooks allow function components to have access to state and other React features. 
* Because of this, class components are generally no longer needed.
* Hooks allow us to "hook" into React features such as state and lifecycle methods.


## useState 

* The React useState Hook allows us to track state in a function component.
* State generally refers to data or properties that need to be tracking in an application.


### step -1  =  Import useState
* At the top of your component, import the `useState` Hook.

```js
import { useState } from "react";
```

### Initialize useState
* We initialize our state by calling useState in our function component.
* useState accepts an initial state and returns two values:
  * The current state.
  * A function that updates the state.
  * `const [variable, setVariable] = useState(variableValue);`
  * `const [currentState, setCurrentState] = useState(currentStateValue);`

```js
import { useState } from "react";

function FavoriteColor() {
  const [color, setColor] = useState("");
}
```

### Update State
* To update our state, we use our state updater function.
  * state updater function :- `setVariable(updatedValue)`

```js
import { useState } from "react";
import ReactDOM from "react-dom/client";

function FavoriteColor() {
  const [color, setColor] = useState("red");

  return (
    <>
      <h1>My favorite color is {color}!</h1>
      <button type="button" onClick={() => setColor("blue")}> Blue </button>
    </>
  )
}

const root = ReactDOM.createRoot(document.getElementById('root'));
root.render(<FavoriteColor />);
```

## useEffect 

## userContext


## userReducer


## userRef


## userMemo


## userCallback


## Custom Hook